---
## - LIST OF STEPS PROMPT

#### Baseline Zero Shot
```py
prompt = """ Provide a detailed list of the Steps in the given Research Software which refers to "[software-name]" plan, including the total count of all the Step? """
```

#### One Shot Learning:

The prompt additionally contains an example instance of a research software installation plan (consisting of a description of the initial step and the end) and the corresponding plan (which ends with a tag, referred to as the plan-end tag, that denotes the end of the plan). The prompt is formatted in natural language (or **JSON/RDF format**?)

**JSON**

```py
prompt = """\nGiven a TEXT, I want you generate task steps and plan type of installation.\
    The format must in a strict JSON format : {[{"plan_type": "detect the method of installation", "task_steps": [a ordered list of steps to install], "commands": [ a concise list of commands for the tool.  \
    If you do not find software installation steps in the TEXT, return "none".\
    Annotate the following TEXT  """
        prompt += """\n\n# REQUIREMENTS #: \n1. the generated task steps and plan types allign with # RESEARCH SOFTWARE TOOL # perfectly. Task name must be detected from the README file; \n2. the task steps should strictly aligned with the plan type, and the number of task steps should be same with the readmes;  \n4.  RESEARCH SOFTWARE TOOL command should be align with the input-type field of # TASK LIST #;"""
```

**ONTOLOGY**

```py

prompt = """ Here is the complete definition of the procedural RDF ontology in TTL format which defines Plans and Steps and relationships between each other:

@prefix p-plan: <http://purl.org/net/p-plan#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix bpmn: <http://www.w3.org/ns/bpmn#> .

[.....]


Using the provided ontology, please create specific instances and data about plans representing the steps and  for the [-] software installation procedure. Include relationships such as 'isStepOfPlan' to model the hierarchy of main steps and their substeps. Additionally, create the RDF graph that represents the defined individuals and their relationships.  Refer to the RDF graph as Context, please provide an Answer to the Question below."""

```

In [1]:
!pip install gpt4all openai -q

**Example for validation ground_true**

```json
            {
                "id": "6",
                "name": "PFischbeck/parameter-fitting-experiments",
                "url": "https://raw.githubusercontent.com/PFischbeck/parameter-fitting-experiments/main/Readme.md",
                "n_plans": 2,
                "plan_nodes": [
                    {
                        "type": "Source",
                        "plan_step": [
                            "Step 1: Make sure you have Python, Pip and R installed.",
                            "Step 2: Checkout this repository.",
                            "Step 3: Install the python dependencies with.",
                            "Step 4: Install the pygirgs package at https://github.com/PFischbeck/pygirgs.",
                            "Step 5: Install the R dependencies (used for plots) with",
                            "Step 6: Download the file at https://doi.org/10.5281/zenodo.10629451 and extract the content",
                            "Step optional: Download the file output-data.zip from Zenodo and extract its contents into the folder output_data. This way, you can access all experiment results without running them yourself."
                        ],
                        "operating_system": []
                    }
                ],
                "readme_instructions": "",
                "skip_content": ""
            },
``````

In [1]:
import json
import openai
import config
api_key = config.API_KEY

import os
import re
import sys
import time
import urllib.request

## 1. - TYPE OF PLANS PROMPT

#### Baseline Zero Shot
```py
prompt_TEXT = """ Given a INSTALL_TEXT, detect the class of plan for the installation of a software."""
prompt_URL =  """ Given a INSTALL_TEXT, detect the class of plan for the installation of a software."""
```

---
**GEMMA: Gemini-based open model**

ref:
[https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf](https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf)

In [35]:
# Given a URL (no labels), Natural language response
from openai import OpenAI
client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key= api_key # your key hf
)
MODEL = "google/gemma-7b-it"

In [29]:
def get_code_completion(messages, max_tokens=512, model=MODEL):
    chat_completion = client.chat.completions.create(
        messages=messages,
        model=model,
        max_tokens=max_tokens,
        # stop=[
        #     "<step>"
        # ],
        # frequency_penalty=1,
        # presence_penalty=1,
        # top_p=0.7,
        # n=10,
        # temperature=1,
    )

    return chat_completion

In [6]:
api_key = config.API_KEY
SYSTEM_PROMPT = "You are a smart and intelligent Named Entity Recognition (NER) system. I will provide you some task to identify entities in a text"
USER_PROMPT_1 = "Detect the TYPE of plan for the installation of a software in the README_INSTALLATION."
ASSISTANT_PROMPT_1 = "Sure, I'm ready to help you with your NER task. Please provide me with the necessary information"
GUIDELINES_PROMPT = (
    "Entity Definition:\n"
    "1. README_INSTALLATION: a excerpt of the text found in a readme file of a research software tool"
    "2. TYPE: represents the concept of a installation method type as a plan, which is composed of steps, which must be executed in a given order. A installation method is an instance of the Plan concept. There are four type of plans: binary, package manager, source and container. A research software readme installation instruction could refer to one or multiple methods\n"
    "3. PLAN_STEP: represents a list of planned action(s) as part of a Plan to be executed by an Agent. These are a list of indivisible sequence of actions that must executed without interruption. Step within a Plan could be linked to one specific executable operation, or refer to a group of operations. A Step then could invoke more than one action. Each sentence in a readme is an instance of the Step concept.\n"
    "4. TECHNOLOGY: repesents the concept of a particular operating sytem or package management correspoding to a particular set of PLAN_STEPs in a specific TYPE.\n"
    "5. N_PLANS: count the number of unique plan TYPE"
    "\n"
    "Output Format:\n"
    "{{'TYPE': [name of the TYPE plans present], 'PLAN_STEP': [list of steps], 'TECHNOLOGY': [name of the technology used for that specific TYPE], 'N_PLANS': [number of unique plan TYPE]}}"
)

In [36]:
from openai import OpenAI
client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key= api_key # your key hf
)
MODEL = "google/gemma-7b-it"

In [37]:
# client = openai.OpenAI()
def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1
    )
    return response.choices[0].message.content

In [47]:
from openai import OpenAI
client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key= api_key # your key hf
)
MODEL = "google/gemma-7b-it"

# client = openai.OpenAI()
def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=1
    )
    return response.choices[0].message.content

readme = f"""
# Install See the [TensorFlow install guide](https://www.tensorflow.org/install) for the [pip package](https://www.tensorflow.org/install/pip), to [enable GPU support](https://www.tensorflow.org/install/gpu), use a [Docker container](https://www.tensorflow.org/install/docker), and [build from source](https://www.tensorflow.org/install/source). To install the current release, which includes support for [CUDA-enabled GPU cards](https://www.tensorflow.org/install/gpu) *(Ubuntu and Windows)*: ``` $ pip install tensorflow ``` Other devices (DirectX and MacOS-metal) are supported using [Device plugins](https://www.tensorflow.org/install/gpu_plugins#available_devices). A smaller CPU-only package is also available: ``` $ pip install tensorflow-cpu ``` To update TensorFlow to the latest version, add `--upgrade` flag to the above commands. *Nightly binaries are available for testing using the [tf-nightly](https://pypi.python.org/pypi/tf-nightly) and [tf-nightly-cpu](https://pypi.python.org/pypi/tf-nightly-cpu) packages on PyPi.
"""
definitions = f"""
1. readme: a excerpt of the text found in a readme file of a research software in Github \
2. Installation Method: indicates a **PLAN** which contains instructions as **STEPS** for installing a research software that must be performed in a precise order and under specific conditions. In the context of the study, there are four general different methods as PLAN_TYPE: 0.source, 1.packagemanager, 2.container and 3.binary. \
3. PLAN_TYPE: represents the concept of an installation method type as a plan, which is composed of steps, which must be executed in a given order. An installation method is an instance of the PLAN concept. A research software readme installation instruction could refer to one or multiple methods as PLAN_TYPE". Here the definition of each TYPE: \
      0.source: Installing a software "from source" means installing a software without using automatic tools e.g. container or package manager.\
      1.Packagemanager: Installing a software "from pip" means installing the software along with its dependencies indexed in official package managers.\
      2.container: provide a way of packaging research software and their dependencies inside lightweight, standalone containers. \
      3.binary: a binary files contains the entire codebase and associated resources needed to run the software, eliminating the need for compilation or building the software from source.
4. STEP: a list of planned activities as part of a `Plan` to be executed in a specific order. Each atomic activity is an instance of the Step concept. A Step within a Plan could comprise one activity, or a group of activities. We define a Step as the readme's author annotated.

    "\n"
"""

prompt = f"""
Given the following \"\"\"{readme}\"\"\", extract the installation instructions for each method (from source, container) or Operative System (Windows, Linux). For each installation method return a list, where each element of the list is an instruction, in sequential order. The response should be a JSON list. Do not add code commands in the list. Be concise.




"""
response = get_completion(prompt)
print("Completion for Readme 1:")
print(response)

Completion for Readme 1:
```json
{
  "from_source": [
    "Build from source",
    "See the [TensorFlow install guide](/install) for the [pip package](/install/pip) and follow the instructions for building from source."
  ],
  "container": [
    "Use a [Docker container](/install/docker) to enable GPU support",
    "See the [TensorFlow install guide](/install) for the [pip package


---
### Generate LLMs response JSON file

In [2]:
# STEP
from openai import OpenAI
import json

client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key=api_key
)
MODEL = "google/gemma-7b-it"




def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
        temperature=0.1
    )
    return response.choices[0].message.content

def extract_installation_instructions_from_json(json_file):
    with open(json_file, 'r') as file:
        readmes = json.load(file)

    results = []

    for readme in readmes:
        readme_instructions = readme.get('readme_instructions', '')
        
        # Use the get_completion function to get the installation instructions
        prompt = f""" 
        You will be provided with a Readme. It contains a sequence of instructions per installation method to install a software \
        There are 4 types of methods which are source, packagemanager, container and binary. \
        Perform the following actions. For each Readme:
         1. Extract the installation instructions for each method  \
         2. For each installation method return a list, where each element of the list is an instruction, in sequential order \
         2. The output should be in JSON format. 
        Do not add code commands in the list. Be concise. \

        Readme:
        ```{readme_instructions}``` 
        """
        prompt += """\n# The format must in a strict JSON format, like: {"plan": [{"type": "installation method in the readme" ], "steps": [{"text": "description of the step per installation instruction", "n_steps": ["count of total steps"]}]} """
        
        response = get_completion(prompt)
        
        # Append the extracted instructions to the results list
        results.append([readme['id'],response])

    return results


json_file_path = '../scr/output.json'
result_list = extract_installation_instructions_from_json(json_file_path)

# # Print the results
# for result in result_list:
#     for key, value in result.items():
#         print(f"{key}:")
#         print(value)
#         print()



with open("step-responses.json", "w") as json_file:
    json.dump(result_list, json_file, indent=2)

print("Responses saved to step-responses.json")


Responses saved to step-responses.json


---

In [10]:
# PLAN

from openai import OpenAI
import json

client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key=api_key
)
MODEL = "google/gemma-7b-it"




def get_completion(prompt, model=MODEL):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        response_format={"type": "json_object"},
        temperature=2
    )
    return response.choices[0].message.content

def extract_installation_instructions_from_json(json_file):
    with open(json_file, 'r') as file:
        readmes = json.load(file)

    results = []

    for readme in readmes:
        readme_instructions = readme.get('readme_instructions', '')
        readme_plans = readme.get('plans', [])

        readme_types = []
        for plan in readme_plans:
            # Extracting the 'type' field from each plan
            readme_type = plan.get('type', '')
            readme_types.append(readme_type)
            
        
        # Use the get_completion function to get the installation instructions
        prompt = f""" 
        Given a  <\"\"\"{readme_instructions}\"\"\", detect the TYPE of PLAN for the installation of a research software. \
        There are 4 TYPE of PLAN: source, packagemanager, container and binary.
        """
        # prompt += """labels: binary, source, packagemanager, container."""
        # prompt += """Classify the following \"\"\"{readme_instructions}\"\"\" in one or multiple installation methods \
        # There are 4 possible installation methods which are: source, packagemanager, container and binary. \
        # A readme can describe a single, or multiple types of installation. \
        # """
        response = get_completion(prompt)
        
        # Append the extracted instructions to the results list
        results.append({"id":readme['id'],"response":response, "answers":readme_types, "prompt":prompt})

    return results


# Example usage:
# Assuming your JSON file has an array of readmes
json_file_path = '../scr/output.json'
result_list = extract_installation_instructions_from_json(json_file_path)

# # Print the results
# for result in result_list:
#     for key, value in result.items():
#         print(f"{key}:")
#         print(value)
#         print()



with open("plan-responses.json", "w") as json_file:
    json.dump(result_list, json_file, indent=2)

print("LLM responses saved to plan-responses.json")


*Other experiments below:>>>>>>>*

In [59]:
messages = [
        {"role": "user", "content": "Behave as an expert labeler. \
          Given a README, detect the class of plan for the installation of a software in the text of the. \n Definition:\n 1. README_INSTALLATION: a excerpt of the text found in a readme file of a research software tool \n 2. TYPE: represents the concept of a installation method type as a plan, which is composed of steps, which must be executed in a given order. A installation method is an instance of the Plan concept. There are four type of plans: binary, package manager, source and container. A research software readme installation instruction could refer to one or multiple methods\n 3. PLAN_STEP: represents a list of planned action(s) as part of a Plan to be executed by an Agent. These are a list of indivisible sequence of actions that must executed without interruption. Step within a Plan could be linked to one specific executable operation, or refer to a group of operations. A Step then could invoke more than one action. Each sentence in a readme is an instance of the Step concept.\n 4. TECHNOLOGY: repesents the concept of a particular operating sytem or package management correspoding to a particular set of PLAN_STEPs in a specific TYPE.\n 5. N_PLANS: count the number of unique plan TYPE \n README = # Installation - Make sure you have Python, Pip and R installed. - Checkout this repository - Install the python dependencies with ```pip3 install -r requirements.txt``` - Install the `pygirgs` package at https://github.com/PFischbeck/pygirgs \ - Install the R dependencies (used for plots) with  ```R -e install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/) Download the file `konect-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `input_data/konect` Optional: Download the file `output-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `output_data`. This way, you can access all experiment results without running them yourself. The Output Format in JSON file:\n {{'TYPE': [name of the TYPE plans present], 'PLAN_STEP': [list of steps], 'TECHNOLOGY': [name of the technology used for that specific TYPE], 'N_PLANS': [number of unique plan TYPE]}} \
         "},
]

chat_completion = get_code_completion(messages)
            
print(chat_completion.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': 'Rate limit reached. You reached free usage limit (reset hourly). Please subscribe to a plan at https://huggingface.co/pricing to use the API at this rate'}

---
**Mistral orca-2 Model**

ref: [mistral-7b-openorca.gguf2.Q4_0.gguf](mistral-7b-openorca.gguf2.Q4_0.gguf)

**One Shot Learning**

**Chain of Thought Prompting**


In addition to the four experiments above, we look at a fifth experiment using a state tracking chain of thought prompting technique in a natural language one shot setting. Within this configuration, we provide an annotated example where each action is annotated with the state prior to the action, the reason for why the action is applicable in the prior state, and the resulting state after applying the action. After the example, a meta-explanation about plan correctness is provided. The LLM is then asked to return a response making the same state tracking and justification annotations that were included in the example

---
**GEMMA: Gemini-based open model**

ref:
[https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf](https://storage.googleapis.com/deepmind-media/gemma/gemma-report.pdf)

**- Baseline Zero Shot**

In [26]:
# Given a URL, Natural language response
messages=[
        {"role": "user", "content": "Behave as an expert labeler. \
          Given a URL, I want you generate a list of steps for the plan of installation. Please indicate the number of the steps. URL = https://raw.githubusercontent.com/PFischbeck/parameter-fitting-experiments/main/Readme.md"},        
    ]
    # stream=True,
    # max_tokens=500

chat_completion = get_code_completion(messages)
            
print(chat_completion.choices[0].message.content)

NameError: name 'get_code_completion' is not defined

In [19]:
# given a URL, JSON object response

chat_completion = client.chat.completions.create(
    model="google/gemma-7b-it",
    messages=[
        {"role": "user", "content": "Behave as an expert labeler. \
          Given a README_TEXT, \
          I want you generate a list of steps for the plan of installation. \
          Please indicate the number of the steps. The RESPONSE Format for your response should be in JSON file as following: \
         \n {{'PLAN_STEP': [`Step 1`: give a number of the step. Each step separated with commas], {{`NUM_STEP`: [count of unique steps]}}}} \n \
         Definition:\n 1. README_INSTALLATION: a excerpt of the text found in a readme file of a research software tool \n 2.PLAN_STEP: represents a list of planned action(s) in sequential orden as part of a installation plan. These are a list of indivisible sequence of actions that must executed without interruption. Step within a Plan could be linked to one specific executable operation, or refer to a group of operations. A Step then could invoke more than one action. \n \
         Here is the README_TEXT = # Installation - Make sure you have Python, Pip and R installed. - Checkout this repository - Install the python dependencies with ```pip3 install -r requirements.txt``` - Install the `pygirgs` package at https://github.com/PFischbeck/pygirgs \ - Install the R dependencies (used for plots) with  ```R -e install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/) Download the file `konect-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `input_data/konect` Optional: Download the file `output-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `output_data`. This way, you can access all experiment results without running them yourself."},
    ],
    stream=True,
    max_tokens=500,
    response_format={"type": "json_object"},
)

for message in chat_completion:
    print(message.choices[0].delta.content, end="")

```json
{
  "PLAN_STEP": ["Step 1: Ensure Python, Pip and R are installed.", "Step 2: Checkout this repository.", "Step 3: Install Python dependencies (e.g. `pip3 install -r requirements.txt`)", "Step 4: Install the `pygirgs` package.", "Step 5: Install R dependencies (used for plots) with  ```R -e install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/)`, "Step 6: Download and extract `konect-data.zip` into `input_data/konect` folder.", "Optional: Download and extract `output-data.zip` into `output_data` folder.],
  "NUM_STEP": 7
}
```

**Number of steps:** 7

**Explanation:**

The README_TEXT describes an installation plan with a total of 7 steps. Each step is clearly defined and described. The steps include ensuring required software dependencies are installed, checking out the repository, installing Python and R dependencies, downloading and extracting data files, and optional steps for downloading additional data.<eos>

In [125]:
# # given a URL, JSON object response
# from openai import OpenAI
# client = OpenAI(
#     base_url="https://api-inference.huggingface.co/v1",
#     api_key= api_key

example_json = {
  "plan_nodes": [
                    {
                        "num_steps": "# total count number of unique steps",
                        "plan_step": [
                            "`# Step 1:` a string to represent the concept of a step in a method or plan. Each atomic activity is an instance of the Step. This step indicates the initial step in a plan",
                            "# `Step 2:` a string to represent the concept of a step in a procedure or plan. It associates a step with the next step that must be performed",
                            "# `Step n:`a string to represent the concept of a step in a procedure or plan. It associates a plan with its last step. This indicates the final step in a plan."
                        ]
                    }
                ]
}


prompt= "Given a , I want you generate a valid JSON output with the list of steps for the plan of installation. \
README_TEXT = # Installation - Make sure you have Python, Pip and R installed. - Checkout this repository - Install the python dependencies with ```pip3 install -r requirements.txt``` - Install the `pygirgs` package at https://github.com/PFischbeck/pygirgs \ - Install the R dependencies (used for plots) with  ```R -e install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/) Download the file `konect-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `input_data/konect` Optional: Download the file `output-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `output_data`. This way, you can access all experiment results without running them yourself."

response = client.chat.completions.create(
    model="google/gemma-7b-it",
    messages=[
        {"role": "user", "content": "Provide output in valid JSON. The data schema should be like this:"+json.dumps(example_json)},
        { "role": "system", "content": prompt}
    ],
    stream=True,
    max_tokens=500,
    response_format={"type": "json_object"},
)
# messages = []
# for message in chat_completion:
#     print(message.choices[0].delta.content, end="")
#     messages.append(message.choices[0].delta.content)

with open('output.json', 'w') as f:
    for chunk in response:
        f.write(chunk.choices[0].delta.content)




In [13]:
# Given a INSTALL_TEXT, JSON object response
from openai import OpenAI
client = OpenAI(
    base_url="https://api-inference.huggingface.co/v1",
    api_key= api_key
)

chat_completion = client.chat.completions.create(
    model="google/gemma-7b-it",
    messages=[
        {"role": "user", "content": "Behave as an expert labeler. \
          Given a INSTALL_TEXT, generate an example json object describing the Steps of installation of the Research Software. Please indicate the total number of the Steps. \
         INSTALL_TEXT = # Installation - Make sure you have Python, Pip and R installed. - Checkout this repository - Install the python dependencies with \
         ```pip3 install -r requirements.txt``` \
         - Install the `pygirgs` package at https://github.com/PFischbeck/pygirgs \
         - Install the R dependencies (used for plots) with \
         ```R -e 'install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/)'``` \
         - Download the file `konect-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `input_data/konect` \
         - Optional: Download the file `output-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `output_data`. This way, you can access all experiment results without running them yourself."},        
    ],
    stream=True,
    max_tokens=500,
    response_format={"type": "json_object"},
)

for message in chat_completion:
    print(message.choices[0].delta.content, end="")

## JSON object for installing research software from the text:

```json
{
  "total_steps": 6,
  "steps": [
    "Make sure you have Python, Pip and R installed.",
    "Checkout this repository",
    "Install the python dependencies with `pip3 install -r requirements.txt`",
    "Install the `pygirgs` package at https://github.com/PFischbeck/pygirgs",
    "Install the R dependencies (used for plots) with `R -e 'install.packages(c(ggplot2, reshape2, plyr, dplyr, scales), repos=https://cloud.r-project.org/)'",
    "Download the file `konect-data.zip` from [Zenodo](https://doi.org/10.5281/zenodo.10629451) and extract its contents into the folder `input_data/konect`. This way, you can access all experiment results without running them yourself. The file `output-data.zip` can be optionally downloaded and extracted into the folder `output_data`."
  ]
}
```

**Total number of steps:** 6<eos>

---

**Mistral orca-2 Model**

ref: [mistral-7b-openorca.gguf2.Q4_0.gguf](mistral-7b-openorca.gguf2.Q4_0.gguf)

In [10]:
from gpt4all import GPT4All
import logging
logging.basicConfig(level=logging.INFO)
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")
with model.chat_session('You are an expert label.\nBe terse.',
                        '### Instruction:\n{0}\n### Response:\n'):
    prompt = ' Given a TEXT, I want you generate task steps and plan type of installation.\
    The format must in a strict JSON format : {[{"plan_type": "detect the method of installation", "task_steps": [a ordered list of steps to install], "commands": [ a concise list of commands for the tool.  \
    If you do not find software installation steps in the TEXT, return "none".\
    Annotate the following TEXT: # Installation ### Dependencies Initialize git submodules with `` git submodule init git submodule update``` \
    Install the specific versions of every package from `requirements.txt` in a new conda environment:\
    conda create --name gsft python=3.9 \
    conda activate gsft\
    pip install -r requirements.txt\
    To ensure that Python paths are properly defined, update the `~/.bashrc` by adding the following lines\
    export GSFT_PATH=/path_to_gsfc\
    export PYTHONPATH=$PYTHONPATH:/$GSFT_PATH'
    print("PROMPT: ", prompt)
    response = model.generate(prompt=prompt, temp=0.6)
    print("Response", response)

INFO:gpt4all._pyllmodel:LLModel.prompt_model -- prompt:
You are an expert label.
Be terse.

### Instruction:
 Given a TEXT, I want you generate task steps and plan type of installation.    The format must in a strict JSON format : {[{"plan_type": "detect the method of installation", "task_steps": [a ordered list of steps to install], "commands": [ a concise list of commands for the tool.      If you do not find software installation steps in the TEXT, return "none".    Annotate the following TEXT: # Installation ### Dependencies Initialize git submodules with `` git submodule init git submodule update```     Install the specific versions of every package from `requirements.txt` in a new conda environment:    conda create --name gsft python=3.9     conda activate gsft    pip install -r requirements.txt    To ensure that Python paths are properly defined, update the `~/.bashrc` by adding the following lines    export GSFT_PATH=/path_to_gsfc    export PYTHONPATH=$PYTHONPATH:/$GSFT_PATH
##

PROMPT:   Given a TEXT, I want you generate task steps and plan type of installation.    The format must in a strict JSON format : {[{"plan_type": "detect the method of installation", "task_steps": [a ordered list of steps to install], "commands": [ a concise list of commands for the tool.      If you do not find software installation steps in the TEXT, return "none".    Annotate the following TEXT: # Installation ### Dependencies Initialize git submodules with `` git submodule init git submodule update```     Install the specific versions of every package from `requirements.txt` in a new conda environment:    conda create --name gsft python=3.9     conda activate gsft    pip install -r requirements.txt    To ensure that Python paths are properly defined, update the `~/.bashrc` by adding the following lines    export GSFT_PATH=/path_to_gsfc    export PYTHONPATH=$PYTHONPATH:/$GSFT_PATH
Response  {[{"plan_type": "software installation", "task_steps": [
"Determine the specific version of 

In [11]:
from gpt4all import GPT4All
model = GPT4All("orca-mini-3b-gguf2-q4_0.gguf")
with model.chat_session():
    prompt = 'Behave as an expert labeler. \ Given a URL, I want you generate task steps and plan type of installation.\
    The format must in a strict JSON format : {"task_steps": [ step description of one or more steps ], "plan_nodes": [{"plan_type": "detect the method of installation", "commands": [ a concise list of commands for the tool. \
    URL: https://raw.githubusercontent.com/PFischbeck/parameter-fitting-experiments/main/Readme.md'
    # print("PROMPT: ", prompt)
    response = model.generate(prompt=prompt, temp=0)
    print("Response", response)

INFO:gpt4all._pyllmodel:LLModel.prompt_model -- prompt:
### System:
You are an AI assistant that follows instruction extremely well. Help as much as you can.

### User:
Behave as an expert labeler. \ Given a URL, I want you generate task steps and plan type of installation.    The format must in a strict JSON format : {"task_steps": [ step description of one or more steps ], "plan_nodes": [{"plan_type": "detect the method of installation", "commands": [ a concise list of commands for the tool.     URL: https://raw.githubusercontent.com/PFischbeck/parameter-fitting-experiments/main/Readme.md
### Response:

===/LLModel.prompt_model -- prompt/===


Response  {
"task_steps": [
 {"step": "Detect the method of installation",
 "commands": ["sudo apt-get update && sudo apt-get install pf-script"]
}
],
"plan_nodes": [
 {"plan_type": "Installation Plan",
 "instructions": [
 "1. Download the latest version of the package manager for your operating system from the official website.",
 "2. Open the terminal and run the command `sudo apt-get update` to check if there are any updates available.",
 "3. Run the command `sudo apt-get install pf-script` to download and install the package manager."
 ]}
]
}


## 2. - TYPE OF PLANS PROMPTS

#### Baseline-  Zero Shot
```py
prompt = """ Classify the text into LABELS [source, binary, container, and package manager]"""
```